<a href="https://colab.research.google.com/github/thisissamuca/GOES-16_GeoTifs_Producs/blob/main/GOES_16_GeoTifs_Producs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 66.6 MB/s eta 0:00:00


In [ ]:
pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.8 MB/s eta 0:00:00


In [ ]:
pip install satpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.3/398.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.4/658.4 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.9/203.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.3 MB/s eta 0:00:00


In [233]:
import glob
import satpy
from satpy import Scene
from satpy.readers import FSFile
import fsspec
import os

In [130]:
# Montando drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Visualizando ops readers disponíveis

from satpy import available_readers
available_readers()

In [236]:
# Criando laços de repetição

# Variavéis fixas

bucket_name = 'noaa-goes16'

product_name = 'ABI-L1b-RadF' # Choose a GOES-R ABI product

#product_name = 'ABI-L2-CMIPF' # Choose a GOES-R ABI product

directory = f"/content/drive/MyDrive/NBR"

# Variáveis dinâmicas

files = []

bands = [3,6]

year = 2022

hour = 17

day_of_year = 152

#day_of_year_final = 212

for day in range(day_of_year, day_of_year+7):

  filename = f'{bucket_name}/{product_name}/{year}/{day}/{hour}/*_G16_s{year}{day}{hour}00*'

  the_files = fsspec.open_files("simplecache::s3://" + filename, s3={'anon': True})

  # the_files = fsspec.open_files("s3://" + filename, anon=True)

  fs_files = [FSFile(open_file) for open_file in the_files]

  files.append(fs_files)

files

[[<FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C01_G16_s20221521700206_e20221521709514_c20221521709560.nc">,
  <FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C02_G16_s20221521700206_e20221521709514_c20221521709556.nc">,
  <FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C03_G16_s20221521700206_e20221521709514_c20221521709567.nc">,
  <FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C04_G16_s20221521700206_e20221521709514_c20221521709541.nc">,
  <FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C05_G16_s20221521700206_e20221521709514_c20221521709570.nc">,
  <FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C06_G16_s20221521700206_e20221521709519_c20221521709564.nc">,
  <FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C07_G16_s20221521700206_e20221521709525_c20221521709575.nc">,
  <FSFile "noaa-goes16/ABI-L1b-RadF/2022/152/17/OR_ABI-L1b-RadF-M6C08_G16_s20221521700206_e2022152170951

In [237]:
for i in range(len(files)):

  scn = Scene(filenames=files[i], reader='abi_l1b')

  for j in range(len(bands)):

    base_filename = os.path.basename(files[i][bands[j]-1])

    output_filename = f"{directory}/{product_name}/{year}/{day_of_year+i}/{base_filename[:-3]}.tif"

    scn.load([f'C0{bands[j]}'])

    scn.save_dataset(f'C0{bands[j]}', filename=output_filename)